In [1]:
import pandas as pd
from plotly import express as px
from Bio import SeqIO

In [2]:
amino_acid_map = {
    'A': 'Ala', 'R': 'Arg', 'N': 'Asn', 'D': 'Asp', 'C': 'Cys',
    'E': 'Glu', 'Q': 'Gln', 'G': 'Gly', 'H': 'His', 'I': 'Ile',
    'L': 'Leu', 'K': 'Lys', 'M': 'Met', 'F': 'Phe', 'P': 'Pro',
    'S': 'Ser', 'T': 'Thr', 'W': 'Trp', 'Y': 'Tyr', 'V': 'Val'
}

In [3]:
fasta = SeqIO.parse("../../seq/NC_005966.1_all_orf.fa", format="fasta")

df_orfs = pd.DataFrame.from_records(
    dict(contig=contig.name, seq=contig.seq)
    for contig in fasta
)
df_orfs

,contig,seq
0,lcl|ORF1_NC_005966.1:898:993,"(M, T, F, T, C, W, R, V, K, K, P, V, W, S, N, ..."
1,lcl|ORF2_NC_005966.1:1264:1377,"(M, W, C, V, N, P, L, K, M, Y, W, R, F, V, R, ..."
2,lcl|ORF3_NC_005966.1:2281:2982,"(M, A, V, Q, D, V, R, F, Y, L, T, G, T, L, L, ..."
3,lcl|ORF4_NC_005966.1:2998:4074,"(M, Q, I, T, R, L, N, I, E, R, V, R, N, L, K, ..."
4,lcl|ORF5_NC_005966.1:6805:6897,"(M, L, A, L, S, S, L, I, I, F, G, K, L, D, F, ..."
...,...,...
326,lcl|ORF327_NC_005966.1:4569:4492,"(M, V, N, C, S, L, L, D, K, A, L, T, T, D, T, ..."
327,lcl|ORF328_NC_005966.1:4458:4357,"(M, V, R, I, T, S, A, A, E, T, P, S, S, G, W, ..."
328,lcl|ORF329_NC_005966.1:4356:4114,"(M, V, T, I, I, S, S, Q, C, P, A, K, A, S, S, ..."
329,lcl|ORF330_NC_005966.1:2943:2863,"(M, C, R, I, L, N, Q, N, R, L, I, G, L, S, H, ..."


In [4]:
df_orfs_exploded = df_orfs.explode('seq')
df_orfs_exploded

,contig,seq
0,lcl|ORF1_NC_005966.1:898:993,M
0,lcl|ORF1_NC_005966.1:898:993,T
0,lcl|ORF1_NC_005966.1:898:993,F
0,lcl|ORF1_NC_005966.1:898:993,T
0,lcl|ORF1_NC_005966.1:898:993,C
...,...,...
330,lcl|ORF331_NC_005966.1:759:628,Q
330,lcl|ORF331_NC_005966.1:759:628,Q
330,lcl|ORF331_NC_005966.1:759:628,P
330,lcl|ORF331_NC_005966.1:759:628,S


In [5]:
df_aa_counts = df_orfs_exploded.seq.value_counts(normalize=False)
df_aa_counts_normalized = df_orfs_exploded.seq.value_counts(normalize=True)

In [13]:
fig_aa_distr = px.bar(
    df_aa_counts_normalized,
    x=df_aa_counts_normalized.index,
    y='proportion',
    color=df_aa_counts_normalized.index.map(lambda x: 'red' if x == 'R' else 'blue'),
    # category_orders={'x': df_aa_counts_normalized.index.tolist()},
    labels={
        'proportion': 'abundance in cDNA',
        'seq': 'amino acid',
    },
).update_layout(
    yaxis=dict(tickformat='.0%'),
    xaxis=dict(
        tickvals=df_aa_counts_normalized.index,
        ticktext=[amino_acid_map[aa] for aa in df_aa_counts_normalized.index]
    ),
    # title='Amino Acids in Putative Genes (by open-reading-frame scan of coding DNA in ADP1)',
    showlegend=False,
    xaxis_categoryorder='total descending',
    template="plotly_white",
)

fig_aa_distr.write_image(
    f'../../figures/arg_abundance_in_cdna.pdf',
    width=800,
    height=300,
)

fig_aa_distr

In [7]:
# It indeed appears that R (arginine is ~5% of putatively expressed AAs) 1/20 = .05